In [1]:
import pandas as pd
import torchtext.data
from torchtext.vocab import build_vocab_from_iterator
from tqdm.notebook import tqdm

# Load CSV files
train_df = pd.read_csv('imdb_train.csv')
test_df = pd.read_csv('imdb_test.csv')

# Define a tokenizer
tokenizer = torchtext.data.get_tokenizer('basic_english')

# Function to iterate over tokens
def iterate_tokens(df):
    for review in tqdm(df['review']):
        yield tokenizer(review)

# Build vocabulary
vocab = build_vocab_from_iterator(
    iterate_tokens(train_df),
    min_freq=5,
    specials=['<unk>', '<s>', '<eos>']
)

vocab.set_default_index(0)

# Print the length of the vocabulary
print(len(vocab))



30124


In [4]:
import torch
from torch.utils.data import (
    TensorDataset,
    DataLoader,
    random_split,
)
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import torchtext.data
from torchtext.vocab import build_vocab_from_iterator
from tqdm.notebook import tqdm

# Load CSV files
train_df = pd.read_csv('imdb_train.csv')
test_df = pd.read_csv('imdb_test.csv')

# Define a tokenizer
tokenizer = torchtext.data.get_tokenizer('basic_english')

# Function to iterate over tokens
def iterate_tokens(df):
    for review in tqdm(df['review']):
        yield tokenizer(review)

# Build vocabulary
vocab = build_vocab_from_iterator(
    iterate_tokens(train_df),
    min_freq=5,
    specials=['<unk>', '<s>', '<eos>']
)

vocab.set_default_index(0)

# Convert text sequences to indices using the vocabulary
sequences = [
    torch.tensor(vocab.lookup_indices(tokenizer(review)), dtype=torch.int64)
    for review in train_df['review']
]

# Pad sequences
padded_sequences = pad_sequence(sequences, batch_first=True)[:, :250]
sentiments = torch.tensor(train_df['sentiment'], dtype=torch.int64)

# Create TensorDataset
dataset = TensorDataset(padded_sequences, sentiments)

# Split dataset into training and validation sets
(train_dataset, val_dataset) = random_split(dataset, (0.7, 0.3))

# Create DataLoader
batch_size = 32
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)


ValueError: Sum of input lengths does not equal the length of the input dataset!

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import torchtext.data
from torchtext.vocab import build_vocab_from_iterator
from tqdm.notebook import tqdm

# Load CSV files
train_df = pd.read_csv('imdb_train.csv')
test_df = pd.read_csv('imdb_test.csv')

# Define a tokenizer
tokenizer = torchtext.data.get_tokenizer('basic_english')

# Function to iterate over tokens
def iterate_tokens(df):
    for review in tqdm(df['review']):
        yield tokenizer(review)

# Build vocabulary
vocab = build_vocab_from_iterator(
    iterate_tokens(train_df),
    min_freq=5,
    specials=['<unk>', '<s>', '<eos>']
)

vocab.set_default_index(0)

# Convert text sequences to indices using the vocabulary
sequences = [
    torch.tensor(vocab.lookup_indices(tokenizer(review)), dtype=torch.int64)
    for review in train_df['review']
]

# Pad sequences
padded_sequences = pad_sequence(sequences, batch_first=True)[:, :250]
sentiments = torch.tensor(train_df['sentiment'], dtype=torch.int64)

# Create TensorDataset
dataset = TensorDataset(padded_sequences, sentiments)

# Calculate lengths for training and validation sets
total_length = len(dataset)
train_length = int(0.7 * total_length)
val_length = total_length - train_length

# Split dataset into training and validation sets
(train_dataset, val_dataset) = random_split(dataset, (train_length, val_length))

# Create DataLoader
batch_size = 32
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)


In [9]:
import torch
import torch.nn as nn
!pip install pytorch_lightning

from pytorch_lightning import LightningModule
from torchmetrics import Accuracy

class MySequenceClassifier(LightningModule):
    def __init__(self, vocab_size, dim_emb, dim_state):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, dim_emb)
        
        self.lstm = nn.LSTM(input_size=dim_emb,
                          hidden_size=dim_state,
                          num_layers=1,
                          batch_first=True)
        
        self.output = nn.Linear(dim_state, 2)
        
        # will be monitoring accuracy
        self.accuracy = Accuracy(task='multiclass', num_classes=2)

In [10]:
import torch.optim
#
# the rest
#
class MySequenceClassifier(MySequenceClassifier):
    def forward(self, seq_batch):
        emb = self.embedding(seq_batch)
        _, (state, _) = self.lstm(emb)
        # state: (num_layers, batch, dim_state)
        output = self.output(state[-1])
        return output
    def loss(self, outputs, targets):
        return nn.functional.cross_entropy(outputs, targets)
    
    def training_step(self, batch, batch_index):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.loss(outputs, targets)
        self.accuracy(outputs, targets)
        self.log('acc', self.accuracy, prog_bar=True)
        self.log('loss', loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    
    def validation_step(self, batch, batch_index):
        inputs, targets = batch
        outputs = self.forward(inputs)
        self.accuracy(outputs, targets)
        self.log('val_acc', self.accuracy, prog_bar=True)

In [12]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

logger = CSVLogger('./lightning_logs/', 'lstm')

trainer = Trainer(max_epochs=10, logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [13]:
model = MySequenceClassifier(vocab_size=len(vocab),
                             dim_emb=32,
                             dim_state=64)

trainer.fit(model,
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader)


  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 963 K 
1 | lstm      | LSTM      | 25.1 K
2 | output    | Linear    | 130   
3 | accuracy  | Accuracy  | 0     
----------------------------------------
989 K     Trainable params
0         Non-trainable params
989 K     Total params
3.957     Total estimated model params size (MB)


C:\Users\baron\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\trainer\data_loading.py:662: UserWarning: Your `val_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.
  category=UserWarning,
C:\Users\baron\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
C:\Users\baron\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number 

C:\Users\baron\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
